# Day 2015_07: Some Assembly Required

In [1]:
year = 2015
day  = 7

In [2]:
from local_settings import load_input
content = load_input(year, day)
print(f"[{content[:100]}...]")

Reading [https://adventofcode.com/2015/day/7/input]
 With proxy: [https: http://proxyseso.scania.com:8080]
 With proxy: [http: http://proxyseso.scania.com:8080]
5304 characters read.
[af AND ah -> ai
NOT lk -> ll
hz RSHIFT 1 -> is
NOT go -> gp
du OR dt -> dv
x RSHIFT 5 -> aa
at OR az...]


# Part 1

In [3]:
def parseCircuit(s):
    for wire in s.splitlines():
        tokens = tuple(wire.split())
        destination = tokens[-1]
        if tokens[0] == 'NOT':
            yield destination, tokens[0], tokens[1], None
        elif tokens[1] == '->':
            yield destination, "ASSIGN", tokens[0], None
        else:
            yield destination, tokens[1], tokens[0], tokens[2]
            
operations = {"AND":lambda x, y:x & y,
              "OR": lambda x, y:x | y,
              "NOT": lambda x, y: 0xFFFF - x,
              "LSHIFT": lambda x, y: (x << y) & 0xFFFF,
              "RSHIFT": lambda x, y: (x >> y) & 0xFFFF,
              "ASSIGN": lambda x, y: x}

def setupCircuit(s):
    circuit = dict()
    for dest, op, arg1, arg2 in parseCircuit(s):
        op = operations[op]
        if arg1.isdecimal():
            arg1 = int(arg1)
        if arg2 is not None and arg2.isdecimal():
            arg2 = int(arg2)
        if dest in circuit:
            print("Overwrite!", dest)
        circuit[dest] = (op, arg1, arg2)
    return circuit

def iterCircuit(circuit, goal):
    while not isinstance(circuit[goal], int):
        nextCircuit = dict()
        for dest, value in circuit.items():
            if isinstance(value, int):
                nextCircuit[dest] = value
                continue
            op, arg1, arg2 = value
            if isinstance(arg1, str):
                if isinstance(circuit[arg1], int):
                    arg1 = circuit[arg1]
            if isinstance(arg2, str):
                if isinstance(circuit[arg2], int):
                    arg2 = circuit[arg2]
            if isinstance(arg1, int) and (arg2 is None or isinstance(arg2, int)):
                nextCircuit[dest] = op(arg1, arg2)
            else:
                nextCircuit[dest] = (op, arg1, arg2)
        circuit = nextCircuit
    return circuit

## Examples:
```
123 -> x
456 -> y
x AND y -> d
x OR y -> e
x LSHIFT 2 -> f
y RSHIFT 2 -> g
NOT x -> h
NOT y -> i
```

In [4]:
example = """123 -> x
456 -> y
x AND y -> d
x OR y -> e
x LSHIFT 2 -> f
y RSHIFT 2 -> g
NOT x -> h
NOT y -> i"""
circuit = setupCircuit(example)
circuit = iterCircuit(circuit, 'i')
for key, value in circuit.items():
    print(f"{key}: {value}")

x: 123
y: 456
d: 72
e: 507
f: 492
g: 114
h: 65412
i: 65079


In [5]:
circuit = setupCircuit(content)
circuit = iterCircuit(circuit, 'a')
print(circuit['a'])

956


# Part 2

## Examples:
```
```

In [6]:
circuit = setupCircuit(content)
circuit['b'] = 956
circuit = iterCircuit(circuit, 'a')
print(circuit['a'])

40149
